# CryptoBench tutorial
This Jupyter Notebook provides basic examples for parsing and exploring the CryptoBench dataset. CryptoBench contains protein structures with cryptic binding sites (BSs). Each ligand-free (**apo**) structure may be linked to multiple ligand-bound (**holo**) structures, as there may be multiple BSs, each associated with different holo structures. 

## Download the dataset
To begin, download the dataset—either manually from [this link](https://osf.io/pz4a9/) or by using the command below.

In [ ]:
!wget -O cryptobench.zip https://files.de-1.osf.io/v1/resources/pz4a9/providers/osfstorage/?zip= --no-check-certificate
!unzip cryptobench.zip
!rm cryptobench.zip

## Initialize the notebook

In [1]:
import json
from typing import TypeAlias, Dict, List

CRYPTOBENCH_PATH = './cryptobench'

JSON: TypeAlias = dict[str, "JSON"] | list["JSON"] | str | int | float | bool | None
BindingSite: TypeAlias = List[str]
ApoPdbId: TypeAlias = str
HoloPdbId: TypeAlias = str
Chain: TypeAlias = str
Ligand: TypeAlias = tuple[str, str, str]

## HOW TO: Load dataset and get APO binding residues
The dataset is provided as a JSON file. The following snippet demonstrates how to extract all **apo** binding residues in a dictionary format, where each key is an apo structure PDB ID and each value is a list of binding residues (annotated with `auth_seq_id`).

In [2]:
def load_dataset(path: str) -> JSON:
    """Loads dataset from JSON file.

    Args:
        path (str): Path to JSON file

    Returns:
        JSON: dataset
    """
    with open(path) as f:
        dataset = json.load(f)
    return dataset


def get_apo_binding_residues(dataset: JSON) -> Dict[ApoPdbId, BindingSite]:
    """Loads binding residues for each APO structure from the dataset. As the APO structure can be associated with more than one HOLO structure, you need to loop over all structures to receive every binding residue.

    Returns:
        Dict[ApoPdbId, BindingSite]: Dictionary of all auth_seq_id indices
    """
    apo_residues = {}
    for apo_pdb_id, holo_structures in dataset.items():
        apo_residues[apo_pdb_id] = set()
        for holo_structure in holo_structures:
            apo_residues[apo_pdb_id].update(holo_structure['apo_pocket_selection'])
    return apo_residues


# you can load the whole dataset or any of the subsets, for example '{CRYPTOBENCH_PATH}/cryptobench-dataset/folds/test.json'
dataset = load_dataset(f'{CRYPTOBENCH_PATH}/cryptobench-dataset/dataset.json')

apo_binding_residues = get_apo_binding_residues(dataset)

## HOW TO: Display cryptic binding residues for Cobyrinic acid a,c diamide synthase (PDB ID: 4pfs)
This enzyme plays a role in the biosynthesis of cobalamin (vitamin B12) in anaerobic bacteria. Binding residues are formatted as `"{auth_asym_id}_{auth_seq_id}"`, where `auth_asym_id` is necessary as some BSs might stretch over multiple chains. 

Let's use the data calculated in the previous example.

In [4]:
APO_STRUCTURE_OF_INTEREST = '4pfs'
apo_binding_residues[APO_STRUCTURE_OF_INTEREST]

{'B_135',
 'B_19',
 'B_192',
 'B_20',
 'B_21',
 'B_22',
 'B_220',
 'B_221',
 'B_222',
 'B_223',
 'B_23',
 'B_24',
 'B_25',
 'B_26',
 'B_48'}

## HOW TO: Get `main` HOLO structures
When analyzing HOLO structures, choosing the representative HOLO for each APO can be difficult. In our CryptoBench manuscript, we selected representative HOLO structures for performance evaluation with P2Rank as follows:

For each APO structure, we chose the HOLO structure that led to inclusion of the respective APO structure in the dataset (e.g., the HOLO with the largest pocket RMSD). These HOLO structures, marked with the `is_main_holo_structure` flag in the dataset, serve as the primary HOLO representatives.

In [5]:
def get_main_holo_structures(dataset: JSON) -> Dict[ApoPdbId, HoloPdbId]:
    """Retrieves 'main' HOLO structure for each APO structure

    Args:
        dataset (JSON): dataset

    Returns:
        Dict[ApoPdbId, HoloPdbId]: apo_pdb_id is key, holo_pdb_id is value
    """
    apo_to_holo = {}
    for apo_pdb_id, holo_structures in dataset.items():
        for holo_structure in holo_structures:
            holo_pdb_id = holo_structure['holo_pdb_id']
            if holo_structure['is_main_holo_structure']:
                apo_to_holo[apo_pdb_id] = holo_pdb_id
        assert apo_pdb_id in apo_to_holo
    return apo_to_holo


main_holo_structures = get_main_holo_structures(dataset)

## HOW TO: Display `main` HOLO structure for Cobyrinic acid a,c diamide synthase (PDB ID: 4pfs)
Let's use the data extracted in the previous example to determine the main HOLO structure for `4pfs`.

In [6]:
# main holo structure for Cobyrinic acid a,c diamide synthase
HOLO_STRUCTURE_OF_INTEREST = main_holo_structures[APO_STRUCTURE_OF_INTEREST] 
HOLO_STRUCTURE_OF_INTEREST

'5ihp'

## HOW TO: Load binding residues for each HOLO structure
Similar to the first snippet, let's extract the binding residues for all HOLO structures. Keep in mind that HOLO structures associated with the same APO structure will be highly similar. Therefore, when analyzing HOLO structures, consider balancing them (e.g., by using the `is_main_holo_structure` flag as demonstrated in the previous example).

In [20]:
def get_holo_binding_residues(dataset: JSON) -> Dict[HoloPdbId, BindingSite]:
    """Get holo binding residues for each HOLO structure in the dataset

    Args:
        dataset (JSON): dataset

    Returns:
        Dict[HoloPdbId, BindingSite]: Dictionary of all HOLO structures, values are their binding residues. 
    """
    holo_residues = {}
    for apo_pdb_id, holo_structures in dataset.items():
        for holo_structure in holo_structures:
            holo_pdb_id = holo_structure['holo_pdb_id']
            if holo_pdb_id not in holo_residues:
                holo_residues[holo_pdb_id] = set()
            holo_residues[holo_pdb_id].update(holo_structure['holo_pocket_selection'])
    return holo_residues

holo_binding_residues = get_holo_binding_residues(dataset)

## HOW TO: Show holo binding residues for `5ihp`
In this example, we will use the data from the previous step to determine the binding residues for the holo structure `5ihp`, which is the **main** HOLO structure for Cobyrinic acid a,c diamide synthase (PDB ID: `4pfs`).

In [21]:
holo_binding_residues[HOLO_STRUCTURE_OF_INTEREST]

{'A_19',
 'A_192',
 'A_20',
 'A_21',
 'A_22',
 'A_220',
 'A_221',
 'A_222',
 'A_223',
 'A_23',
 'A_24',
 'A_25',
 'A_26'}

## HOW TO: get ligands from particular pair 
How to retrieve ligand information from the `4pfs-5ihp` pair. The `5ihp` structure binds `ADP` and can be found in chain `A`, index `1001`.

In [24]:
def get_ligand_information(dataset: JSON, apo_pdb_id: ApoPdbId, holo_pdb_id: HoloPdbId) -> List[Ligand]:
    """Retrieve information about ligand in apo-holo pair. The ligand is present only in the HOLO form, therefore the information is applicable only for the HOLO structure as there is no ligand in APO structure.

    Args:
        dataset (JSON): dataset
        apo_pdb_id (ApoPdbId): APO structure
        holo_pdb_id (HoloPdbId): HOLO structure

    Returns:
        List[Ligand]: List of ligand acronyms, ligand auth_seq_ids, and ligand auth_asym_ids
    """
    assert apo_pdb_id in dataset, f'{apo_pdb_id} is not present in the dataset'
    ligands = []
    for holo_structure in dataset[apo_pdb_id]:
        this_holo_pdb_id = holo_structure['holo_pdb_id']
        this_ligand = (holo_structure['ligand'], holo_structure['ligand_index'], holo_structure['ligand_chain'])
        if this_holo_pdb_id == holo_pdb_id:
            ligands.append(this_ligand)
    return ligands       

get_ligand_information(dataset, APO_STRUCTURE_OF_INTEREST, HOLO_STRUCTURE_OF_INTEREST)

[('ADP', '1001', 'A')]

## HOW TO: Retrieve all apo-holo pairs from CryptoBench
Let’s retrieve all apo-holo pairs as a set of tuples, each containing an APO and a corresponding HOLO PDB ID. Note that a single APO structure may be linked to multiple HOLO structures.

In [30]:
def get_apo_holo_pairs(dataset: JSON) -> set[tuple[ApoPdbId, HoloPdbId]]:
    """Retrieves every apo-holo pair from the dataset

    Args:
        dataset (JSON): dataset

    Returns:
        set[tuple[ApoPdbId, HoloPdbId]]: every apo-holo pair
    """
    apo_holo_pairs = set()
    for apo_pdb_id, holo_structures in dataset.items():
        for holo_structure in holo_structures:
            holo_pdb_id = holo_structure['holo_pdb_id'] 
            apo_holo_pairs.add((apo_pdb_id, holo_pdb_id))
    return apo_holo_pairs

apo_holo_pairs = get_apo_holo_pairs(dataset)

## HOW TO: Display pairs associated with the `4pfs` APO structure
Using the data from the previous snippet, let’s find all HOLO structures linked to `4pfs`. Additionally, display ligand information for each associated HOLO structure.

In [57]:
selected_apo_holo_pairs = [(apo, holo) for apo, holo in apo_holo_pairs if apo == APO_STRUCTURE_OF_INTEREST]
selected_ligands = [get_ligand_information(dataset, apo, holo) for apo, holo in selected_apo_holo_pairs]
print('APO-HOLO pairs asociated with "4pfs" APO structure: ', selected_apo_holo_pairs)
print('Ligands in each HOLO structure: ', selected_ligands)

APO-HOLO pairs asociated with "4pfs" APO structure:  [('4pfs', '5if9'), ('4pfs', '5ihp')]
Ligands in each HOLO structure:  [[('ANP', '1001', 'B')], [('ADP', '1001', 'A')]]


## HOW TO: Determine if two ligands bind to the same BS
In the `4pfs` APO structure, two ligands, `ANP` and `ADP`, appear to bind. To see if both ligands occupy the same BS, let’s check whether their sets of binding residues overlap significantly.

If we observe a substantial overlap, it would indicate a single *promiscuous* BS in `4pfs` capable of binding both ligands. By contrast, if there is minimal or no overlap, it suggests that `ANP` and `ADP` bind at two distinct binding sites.

In [60]:
def get_binding_site_in_apo(dataset: JSON, apo_pdb_id: ApoPdbId, holo_pdb_id: HoloPdbId, ligand: Ligand) -> BindingSite:
    holo_structures = dataset[apo_pdb_id]
    for holo_structure in holo_structures:
        this_holo_pdb_id = holo_structure['holo_pdb_id']
        this_ligand = (holo_structure['ligand'], holo_structure['ligand_index'], holo_structure['ligand_chain'])
        if this_holo_pdb_id == holo_pdb_id and ligand == this_ligand:
            return holo_structure['apo_pocket_selection']
    assert False, "Specified binding site doesn't exist"

def are_binding_sites_promiscuous(binding_site1: BindingSite, binding_site2: BindingSite, threshold=0.75) -> bool:
    """Check whether two binding sites are overlaping more than certain threshold. True value indicates that the two selections are actually indicating a single promiscuous binding site (binding site capable to bind more than one type of ligand)

    Args:
        binding_site1 (BindingSite): selection of first binding site
        binding_site2 (BindingSite): selection of second binding site
        threshold (float, optional): similarity threshold. Defaults to 0.75.

    Returns:
        bool: True if the binding sites are overlaping significantly.
    """
    overlap_length = len(list(set(binding_site1) & set(binding_site2)))
    if overlap_length / len(binding_site1) > threshold and overlap_length / len(binding_site2) > threshold:
        return True
    return False

selected_ligands = [get_ligand_information(dataset, apo, holo) for apo, holo in selected_apo_holo_pairs]
# we already know that there is only one ligand in each HOLO structure so we can do this:
selected_ligands = [i[0] for i in selected_ligands]

binding_site1: BindingSite = get_binding_site_in_apo(dataset, selected_apo_holo_pairs[0][0], selected_apo_holo_pairs[0][1], selected_ligands[0])
binding_site2: BindingSite = get_binding_site_in_apo(dataset, selected_apo_holo_pairs[1][0], selected_apo_holo_pairs[1][1], selected_ligands[1])

print(f'Are the two selections overlaping significantly: {are_binding_sites_promiscuous(binding_site1, binding_site2)}')


Are the two selections overlaping significantly: True


## HOW TO: Extract BSs spanning multiple chains
CryptoBench includes examples where binding sites span across multiple chains. Let’s retrieve all such multichain binding sites. Then, we’ll display the binding residues for the multichain binding site in `1q5v`.

In [4]:
def get_multichain_apo_binding_sites(dataset: JSON) -> Dict[ApoPdbId, BindingSite]:
    """Get BSs from CryptoBench APO structures which stretch over multiple chains

    Args:
        dataset (JSON): dataset

    Returns:
        Dict[ApoPdbId, BindingSite]: multichain binding sites
    """
    multichain_binding_sites = {}
    for apo_pdb_id, holo_structures in dataset.items():
        for holo_structure in holo_structures:
            apo_chain_id = holo_structure['apo_chain'] 
            holo_chain_id = holo_structure['holo_chain'] 
            if '-' in apo_chain_id or '-' in holo_chain_id:
                if apo_pdb_id not in multichain_binding_sites: multichain_binding_sites[apo_pdb_id] = set(holo_structure['apo_pocket_selection'])
                else: multichain_binding_sites[apo_pdb_id].update(holo_structure['apo_pocket_selection'])
    return multichain_binding_sites

multichain_binding_sites = get_multichain_apo_binding_sites(dataset)
multichain_binding_sites['1q5v'] # has 8 binding residues in 'A' chain and 4 binding residues in 'B' chain 

{'A_19',
 'A_20',
 'A_23',
 'A_25',
 'A_31',
 'A_34',
 'A_35',
 'A_38',
 'B_124',
 'B_125',
 'B_40',
 'B_43'}

## Unzip the CIF files
To proceed, you’ll need to unzip the CIF files from the ZIP file included in the dataset. Additionally, make sure the `biopython` package is installed. If not, install it by running the command `pip install biopython`.

In [ ]:
!unzip {CRYPTOBENCH_PATH}/cryptobench-dataset/auxiliary-data/cif-files.zip -d {CRYPTOBENCH_PATH}/cryptobench-dataset/auxiliary-data
!rm {CRYPTOBENCH_PATH}/cryptobench-dataset/auxiliary-data/cif-files.zip

In [13]:
!python3 -m pip install --upgrade biopython

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for biopython from https://files.pythonhosted.org/packages/4c/3c/cecf231afa65e7194ac06ba981631a9870515bb7a37a15cad1ab414325c4/biopython-1.84-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: biopython
    Found existing installation: biopython 1.81
    Uninstalling biopython-1.81:
      Successfully uninstalled biopython-1.81

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## HOW TO: Convert `auth_seq_ids` to `label_seq_ids`
In PDB files, the author-assigned residue numbers (`auth_seq_id`)  may differ from the sequential numbering assigned by PDB (`label_seq_id`). In CryptoBench, binding residues are identified using author-defined residue numbers.

Let’s use the `4n5g` structure as an example, because the `auth_seq_ids` and `label_seq_ids` do not match. Here’s how to translate `auth_seq_ids` to `label_seq_ids` for this structure.

In [25]:
from Bio.PDB.MMCIFParser import MMCIFParser
def auth_to_label(pdb_id: ApoPdbId, chain_id: Chain, binding_site: BindingSite) -> BindingSite:

    # binding residues have the following format: 'A_11'. We need to extract the numbers:
    auth_binding_site = [int(i.split('_')[1]) for i in binding_site]

    parser = MMCIFParser(QUIET=True)
    auth_structure = parser.get_structure(pdb_id, f"{CRYPTOBENCH_PATH}/cryptobench-dataset/auxiliary-data/cif-files/{pdb_id.lower()}.cif")

    parser = MMCIFParser(auth_residues=False, QUIET=True)
    label_structure = parser.get_structure(pdb_id, f"{CRYPTOBENCH_PATH}/cryptobench-dataset/auxiliary-data/cif-files/{pdb_id.lower()}.cif")

    label_binding_site = set()
    for auth_residue, label_residue in zip(auth_structure[0][chain_id].get_residues(), label_structure[0][chain_id].get_residues()):
        if auth_residue.get_id()[0][0] == ' ':
            auth_seq_id = auth_residue.get_id()[1]
            label_seq_id = label_residue.get_id()[1]

            if auth_seq_id in auth_binding_site:
                label_binding_site.add(label_seq_id)

    return label_binding_site

pdb_id = '4n5g'
binding_residues = list(apo_binding_residues[pdb_id])
chain_id = binding_residues[0].split('_')[0]
# check that the BS in `APO_STRUCTURE_OF_INTEREST` is not stretching over multiple chains
for binding_residue in binding_residues:
    assert chain_id == binding_residue.split('_')[0]
label_binding_residues = auth_to_label(pdb_id, chain_id, binding_residues)

print("Let's compare the outputs:")
print('annotated by auth_seq_id:\t', sorted({int(i.split('_')[1]) for i in binding_residues}))
print('annotated by label_seq_id:\t', sorted(label_binding_residues))


Let's compare the outputs:
annotated by auth_seq_id:	 [265, 268, 269, 271, 272, 275, 276, 305, 306, 309, 310, 313, 316, 324, 325, 326, 327, 328, 332, 337, 342, 345, 346, 349, 432, 433, 435, 436, 438, 439, 441, 442, 451]
annotated by label_seq_id:	 [47, 50, 51, 53, 54, 57, 58, 87, 88, 91, 92, 95, 98, 106, 107, 108, 109, 110, 114, 119, 124, 127, 128, 131, 214, 215, 217, 218, 220, 221, 223, 224, 233]


## HOW TO: Extract sequence from an mmCIF file with binding residue indices
When developing sequence-based methods, it can be helpful to extract a structure’s sequence along with binding residue indices within that sequence (`label_seq_id`/`auth_seq_id` may not be reliable due to unobserved residues). Here, we’ll demonstrate how to retrieve the sequence and binding residue indices directly from an mmCIF file.

In [33]:
from Bio.Data.IUPACData import protein_letters_3to1

def get_sequence_with_indices(pdb_id: ApoPdbId, chain_id: Chain, binding_site: BindingSite) -> tuple[str, BindingSite]:

    # binding residues have the following format: 'A_11'. We need to extract the numbers:
    binding_site = [int(i.split('_')[1]) for i in binding_site]

    parser = MMCIFParser(QUIET=True)
    auth_structure = parser.get_structure(pdb_id, f"{CRYPTOBENCH_PATH}/cryptobench-dataset/auxiliary-data/cif-files/{pdb_id.lower()}.cif")

    sequence = ''
    sequence_binding_site = set()
    counter = 0
    for residue in auth_structure[0][chain_id].get_residues():
        if residue.get_id()[0][0] == ' ':
            sequence += protein_letters_3to1[residue.get_resname().title()]

            seq_id = residue.get_id()[1]
            if seq_id in binding_site:
                sequence_binding_site.add(counter)
            counter += 1

    return sequence, sequence_binding_site

sequence, sequence_binding_site = get_sequence_with_indices(pdb_id, chain_id, binding_residues)
print(f'sequence:\t{sequence}')
print('indices of binding residues with corresponding amino acid:\t', [f'{sequence[i]}{i}' for i in sorted(sequence_binding_site)])

sequence:	PVERILEAELAPNDPVTNICQAADKQLFTLVEWAKRIPHFSELPLDDQVILLRAGWNELLIASFSHRSIAVKDGILLATGLHVHRNSAHSAGVGAIFDRVLTELVSKMRDMQMDKTELGCLRAIVLFNPDSKGLSNPAEVEALREKVYASLEAYCKHKYPEQPGRFAKLLLRLPALRSIGLKCLEHLFFFKLIGDTPIDTFLMEMLEAP
indices of binding residues with corresponding amino acid:	 ['V15', 'I18', 'C19', 'A21', 'A22', 'Q25', 'L26', 'W55', 'N56', 'L59', 'I60', 'F63', 'R66', 'I74', 'L75', 'L76', 'A77', 'T78', 'V82', 'A87', 'V92', 'I95', 'F96', 'V99', 'C182', 'L183', 'H185', 'L186', 'F188', 'F189', 'L191', 'I192', 'L201']


## HOW TO: Extract the non-cryptic BSs from the CryptoBench
CryptoBench includes binding sites that did not meet crypticity criteria (stored separately in `cryptobench-dataset/auxiliary-data/non-cryptic-pockets/noncryptic-pockets.json`). Let’s retrieve these non-cryptic BSs using previous code snippets and explore two examples:
1. A non-cryptic BS that is distinct from the cryptic binding site in the `1bhs` APO structure.
2. Two overlapping binding site selections from the `3i8s` APO structure — one from the non-cryptic set and the other from the main (cryptic) CryptoBench set. Both selections bind the **same** ligand and represent the same site. However, we have two snapshots of this binding: one in a more flexible state (`3i8sB-3i8xA` apo-holo pair) and the other in a less flexible state (`3i8sB-3i8xC` apo-holo pair).

In [24]:
noncryptic_dataset = load_dataset(f'{CRYPTOBENCH_PATH}/cryptobench-dataset/auxiliary-data/non-cryptic-pockets/noncryptic-pockets.json')
apo_noncryptic_binding_residues = get_apo_binding_residues(noncryptic_dataset)


print("Two DISTINCT binding sites:")
print(f"cryptic BS: {sorted(list(apo_binding_residues['3pbf']), key=lambda x: int(x.split('_')[1]))}")
print(f"non-cryptic BS:{sorted(list(apo_noncryptic_binding_residues['3pbf']), key=lambda x: int(x.split('_')[1]))}\n")

print("Two binding site selections with significant overlap, suggesting a single binding site observed in two states: one more flexible and the other less flexible:")
print(f"cryptic BS: {sorted(list(apo_binding_residues['3i8s']), key=lambda x: int(x.split('_')[1]))}")
print(f"non-cryptic BS:{sorted(list(apo_noncryptic_binding_residues['3i8s']), key=lambda x: int(x.split('_')[1]))}")

# overlapping 1arl-1bav ;;; 1rjb, 2w8n
# distinct BSs 1bhs-1a27 ;;;1pt7

Two DISTINCT binding sites:
cryptic BS: ['A_128', 'A_129', 'A_132', 'A_172', 'A_203', 'A_218', 'A_219']
non-cryptic BS:['A_141', 'A_142', 'A_144', 'A_146', 'A_168', 'A_180', 'A_181', 'A_182', 'A_227']

Two binding site selections with significant overlap, suggesting a single binding site observed in two states: one more flexible and the other less flexible:
cryptic BS: ['B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_120', 'B_121', 'B_123', 'B_124', 'B_148', 'B_149', 'B_150', 'B_151']
non-cryptic BS:['B_11', 'B_12', 'B_120', 'B_121', 'B_122', 'B_123', 'B_124', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18']
